In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.
record_limit = '999'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp, pformat
from urllib.parse import urlencode
from IPython.display import FileLink, display_markdown
from matplotlib import pyplot as plt

In [ ]:
env = 'usdf_dev'  # usdf-dev, tucson, slac, summit
log_name = 'narrativelog'
log = log_name
limit = int(record_limit)
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds

timeout = (float(response_timeout), float(read_timeout))

# Env list comes from drop-down menu top of:
# https://rsp.lsst.io/v/usdfdev/guides/times-square/
envs = dict(
    #rubin_usdf_dev = '',
    #data_lsst_cloud = '',
    #usdf = '',
    #base_data_facility = '',
    summit = 'https://summit-lsp.lsst.codes',
    usdf_dev = 'https://usdf-rsp-dev.slac.stanford.edu',
    #rubin_idf_int = '',
    tucson = 'https://tucson-teststand.lsst.codes',
)
server = envs[env]
service = f'{server}/{log}'
service

<a class="anchor" id="setup_source"></a>
## Setup Source

In [ ]:
md = f'### Will retrieve from {service}'
display_markdown(md, raw=True)

In [ ]:
recs = None
ok = True

# is_human=either&is_valid=either&offset=0&limit=50' 
# site_ids=tucson&message_text=wubba&min_level=0&max_level=999&user_ids=spothier&user_agents=LOVE
# tags=love&exclude_tags=ignore_message
qparams = dict(is_human='either',
               is_valid='either',
               limit=limit,
              )
qstr = urlencode(qparams)
url = f'{service}/messages?{qstr}'

ignore_fields = set(['tags', 'urls', 'message_text', 'id', 'date_added', 
                     'obs_id', 'day_obs', 'seq_num', 'parent_id', 'user_id',
                     'date_invalidated', 'date_begin', 'date_end',
                     'time_lost', # float
                     #'systems','subsystems','cscs',  # values are lists, special handling
                    ])

In [ ]:
display_markdown(f'## Get (up to {limit}) Records', raw=True)

In [ ]:
# TODO Often fails on first request.  Find out why!
try:
    response = requests.get(url, timeout=timeout)
except:
    pass 
    
try:
    print(f'Attempt to get logs from {url=}')
    response = requests.get(url, timeout=timeout)
    response.raise_for_status()
    recs = response.json()
    flds = set(recs[0].keys())
    facflds = flds - ignore_fields
    # facets(field) = set(value-1, value-2, ...)
    facets = {fld: set([str(r[fld])
                for r in recs if not isinstance(r[fld], list)]) 
                    for fld in facflds}
except Exception as err:
    ok = False
    print(f'ERROR getting {log} from {env=} using {url=}: {err=}')
numf = len(flds) if ok else 0
numr = len(recs) if ok else 0
print(f'Retrieved {numr} records, each with {numf} fields.')

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

### Fields names provided in records from log.

In [ ]:
pd.DataFrame(flds, columns=['Field Name'])

### Facets from log records.
A *facet* is the set all of values found for a field in the retrieved records. Facets are only calculated for some fields.

In [ ]:
display(pd.DataFrame.from_dict(facets, orient='index'))
display(facets)

### Table of selected log record fields.
Table can be retrieved as CSV file for local use.

In [ ]:
cols = ['date_added', 'time_lost']
df = pd.DataFrame(recs)[cols]

# Allow download of CSV version of DataFrame
csvfile = 'tl.csv'
df.to_csv(csvfile)
myfile = FileLink(csvfile)
print('Table available as CSV file: ')
display(myfile)
df

In [ ]:
df = pd.DataFrame(recs)
df

<a class="anchor" id="plot"></a>
## Plots from log

In [ ]:
x = [r['date_added'] for r in recs]
y = [r['time_lost'] for r in recs]
plt.plot(x, y) 
plt.show()

<a class="anchor" id="raw_analysis"></a>
## Raw Content Analysis

### Example of one record

In [ ]:
display(recs[-1])

In [ ]:
msg = rec["message_text"]
md = f'Message text from log:\n> {msg}'
display_markdown(md, raw=True)

<a class="anchor" id="elicitation"></a>
## Stakeholder Elicitation